# Notebook 2: Restrições de Riqueza e Bandas de Preço

Este notebook explora como **wealth limits** (restrições de capital e holdings) transformam a dinâmica do mercado.

## Objetivos de Aprendizado

1. Entender como restrições de riqueza limitam a participação no mercado
2. Observar a formação de bandas de preço naturais
3. Comparar mercados com e sem wealth limits
4. Analisar matematicamente a emergência de bandas

In [ ]:
# Imports
import random
import numpy as np
from market_lab.core.market import MarketConfig
from market_lab.core.traders import build_traders
from market_lab.core.sentiment import NoSentiment
from market_lab.core.simulation import SimulationRunner
from market_lab.viz.plots import plot_price_series

import matplotlib.pyplot as plt
%matplotlib inline

## Experimento 1: Random Walk (sem limites)

Primeiro, vamos simular um mercado **sem restrições de riqueza** como baseline.

In [ ]:
# Configuração do mercado SEM wealth limits
config_unlimited = MarketConfig(
    n_traders=150,
    initial_price=100.0,
    price_volatility=2.0,
    max_daily_volume=10.0,
    wealth_mode="unlimited",
    price_tick=1.0,
    seed=42
)

# Criar traders sem limites
rng = random.Random(42)
traders_unlimited = build_traders(config_unlimited, rng)

# Executar simulação
runner_unlimited = SimulationRunner(
    config=config_unlimited,
    traders=traders_unlimited,
    sentiment=NoSentiment(),
    manipulator=None
)

states_unlimited = runner_unlimited.run(n_days=200)
print(f"Simulação SEM limites: {len(states_unlimited)} dias")

## Experimento 2: Wealth-Limited Market

Agora vamos simular o mesmo mercado **com restrições de riqueza**.

In [ ]:
# Configuração do mercado COM wealth limits
config_limited = MarketConfig(
    n_traders=150,
    initial_price=100.0,
    price_volatility=2.0,
    max_daily_volume=10.0,
    wealth_mode="limited",
    price_tick=1.0,
    seed=42
)

# Criar traders com limites
rng = random.Random(42)
traders_limited = build_traders(config_limited, rng)

# Executar simulação
runner_limited = SimulationRunner(
    config=config_limited,
    traders=traders_limited,
    sentiment=NoSentiment(),
    manipulator=None
)

states_limited = runner_limited.run(n_days=200)
print(f"Simulação COM limites: {len(states_limited)} dias")

## Comparação Visual

Vamos comparar as duas séries de preço lado a lado.

In [ ]:
# Comparar séries de preço
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Mercado sem limites
plot_price_series(states_unlimited, ax=ax1)
ax1.set_title('Random Walk (sem wealth limits)')
ax1.set_ylim(50, 150)

# Mercado com limites
plot_price_series(states_limited, ax=ax2)
ax2.set_title('Wealth-Limited Market (com limites)')
ax2.set_ylim(50, 150)

plt.tight_layout()
plt.show()

# Estatísticas comparativas
prices_unlimited = [s.price for s in states_unlimited]
prices_limited = [s.price for s in states_limited]

print("\nEstatísticas Comparativas:")
print(f"\nSEM limites:")
print(f"  Preço médio: {np.mean(prices_unlimited):.2f}")
print(f"  Desvio padrão: {np.std(prices_unlimited):.2f}")
print(f"  Min/Max: {min(prices_unlimited):.2f} / {max(prices_unlimited):.2f}")

print(f"\nCOM limites:")
print(f"  Preço médio: {np.mean(prices_limited):.2f}")
print(f"  Desvio padrão: {np.std(prices_limited):.2f}")
print(f"  Min/Max: {min(prices_limited):.2f} / {max(prices_limited):.2f}")

## Análise: Por que aparecem bandas?

### Mecânica das Restrições

Quando traders têm **wealth limits**:

1. **Limite de compra**: Traders não podem comprar mais do que `wealth / price`
2. **Limite de venda**: Traders não podem vender mais do que seus `holdings`

### Consequência: Mean Reversion

- **Preço sobe muito**: Traders ficam sem capital para comprar → pressão de venda aumenta
- **Preço cai muito**: Traders ficam sem holdings para vender → pressão de compra aumenta

Isso cria **bandas naturais** ao redor do preço inicial.

## Análise de Volatilidade

Vamos comparar a volatilidade realizada dos dois mercados.

In [ ]:
# Calcular retornos
returns_unlimited = [(prices_unlimited[i] - prices_unlimited[i-1]) / prices_unlimited[i-1] * 100 
                     for i in range(1, len(prices_unlimited))]
returns_limited = [(prices_limited[i] - prices_limited[i-1]) / prices_limited[i-1] * 100 
                   for i in range(1, len(prices_limited))]

# Visualizar distribuições
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogramas
ax1.hist(returns_unlimited, bins=30, alpha=0.7, label='Sem limites', edgecolor='black')
ax1.hist(returns_limited, bins=30, alpha=0.7, label='Com limites', edgecolor='black')
ax1.set_xlabel('Retorno (%)')
ax1.set_ylabel('Frequência')
ax1.set_title('Distribuição dos Retornos')
ax1.legend()

# Rolling volatility
window = 20
vol_unlimited = [np.std(returns_unlimited[max(0, i-window):i+1]) 
                 for i in range(len(returns_unlimited))]
vol_limited = [np.std(returns_limited[max(0, i-window):i+1]) 
               for i in range(len(returns_limited))]

ax2.plot(vol_unlimited, label='Sem limites', alpha=0.7)
ax2.plot(vol_limited, label='Com limites', alpha=0.7)
ax2.set_xlabel('Dia')
ax2.set_ylabel('Volatilidade (rolling 20d)')
ax2.set_title('Volatilidade Realizada')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"\nVolatilidade média:")
print(f"  Sem limites: {np.std(returns_unlimited):.4f}%")
print(f"  Com limites: {np.std(returns_limited):.4f}%")

## Visualização das Bandas

Vamos plotar o preço com bandas calculadas empiricamente.

In [ ]:
# Calcular bandas empíricas (média ± 2 std)
mean_price = np.mean(prices_limited)
std_price = np.std(prices_limited)
upper_band = mean_price + 2 * std_price
lower_band = mean_price - 2 * std_price

# Plotar com bandas
plt.figure(figsize=(14, 6))
days = [s.day for s in states_limited]
plt.plot(days, prices_limited, label='Preço', linewidth=2)
plt.axhline(y=mean_price, color='green', linestyle='--', label=f'Média: {mean_price:.2f}')
plt.axhline(y=upper_band, color='red', linestyle='--', label=f'Upper band: {upper_band:.2f}')
plt.axhline(y=lower_band, color='red', linestyle='--', label=f'Lower band: {lower_band:.2f}')
plt.fill_between(days, lower_band, upper_band, alpha=0.2, color='gray')
plt.xlabel('Dia')
plt.ylabel('Preço')
plt.title('Wealth-Limited Market com Bandas Empíricas')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Calcular % do tempo dentro das bandas
inside_bands = sum(1 for p in prices_limited if lower_band <= p <= upper_band)
pct_inside = (inside_bands / len(prices_limited)) * 100
print(f"\nTempo dentro das bandas: {pct_inside:.1f}%")

## Análise Matemática: Por que ±2σ?

As bandas aparecem porque:

1. **Initial endowment**: Traders começam com capital e holdings limitados
2. **Constraint binding**: Quando preço desvia muito, restrições se tornam ativas
3. **Mean reversion**: Pressão assimétrica força retorno à média

A largura das bandas depende de:
- Distribuição inicial de riqueza
- Volatilidade configurada (`price_volatility`)
- Número de traders

## Exercícios Práticos

### Exercício 1: Riqueza Inicial
Modifique a distribuição inicial de riqueza dos traders:
- O que acontece se todos têm a mesma riqueza inicial?
- E se houver mais desigualdade (alguns muito ricos, outros pobres)?

### Exercício 2: Volatilidade
Aumente `price_volatility` de 2.0 para 5.0:
- As bandas ficam mais largas?
- O mercado permanece mean-reverting?

### Exercício 3: Número de Traders
Compare mercados com 50, 150, e 300 traders:
- Mais traders criam bandas mais estreitas?
- A liquidez afeta a estabilidade?

### Exercício 4: Teste de Mean Reversion
Calcule a autocorrelação dos retornos:
- Mercados com wealth limits têm autocorrelação negativa?
- Isso viola eficiência de mercado?

In [ ]:
# Exemplo de solução para Exercício 4: Mean Reversion Test
lags = range(1, 21)
autocorr_limited = [np.corrcoef(returns_limited[:-lag], returns_limited[lag:])[0, 1] 
                    for lag in lags]
autocorr_unlimited = [np.corrcoef(returns_unlimited[:-lag], returns_unlimited[lag:])[0, 1] 
                      for lag in lags]

plt.figure(figsize=(12, 5))
plt.bar(np.array(lags) - 0.2, autocorr_unlimited, width=0.4, alpha=0.7, label='Sem limites')
plt.bar(np.array(lags) + 0.2, autocorr_limited, width=0.4, alpha=0.7, label='Com limites')
plt.xlabel('Lag (dias)')
plt.ylabel('Autocorrelação')
plt.title('Teste de Mean Reversion')
plt.axhline(y=0, color='r', linestyle='--')
plt.axhline(y=1.96/np.sqrt(len(returns_limited)), color='gray', linestyle='--', label='95% CI')
plt.axhline(y=-1.96/np.sqrt(len(returns_limited)), color='gray', linestyle='--')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print("\nInterpretação:")
print("- Autocorrelação negativa em lag=1 indica mean reversion")
print("- Mercados com wealth limits tendem a reverter à média")
print("- Isso cria oportunidades de arbitragem (ineficiência)")

## Conclusões

Neste notebook, você aprendeu:

1. **Wealth limits transformam mercados**: De random walk para mean-reverting
2. **Bandas emergem naturalmente**: Como resultado de restrições de capital
3. **Mean reversion é detectável**: Via autocorrelação negativa
4. **Implicações práticas**: Mercados reais têm restrições → ineficiências exploráveis

### Próximos Passos

No **Notebook 3**, vamos explorar como manipuladores exploram essas dinâmicas para executar ataques pump-and-dump.